In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
# Define the system of ODEs
def omega0(k0, astar):
    sqrt_term = 4*(1-k0)**2-4*astar*k0*(1-k0)-astar*k0**2
    denom = 2*np.sqrt(astar)
    return np.sqrt(sqrt_term)/denom

def k(t, k0, k1, n, astar):
    return k0 + k1*np.cos(omega0(k0, astar)*t/n)

def l(t, alpha, k0, k1, n, astar):
    l1 = (1/astar)*(1-k(t, k0, k1, n, astar))-k(t, k0, k1, n, astar)
    l0 = (1/astar)*(1-k0)-k0
    return alpha*l1 + (1-alpha)*l0

def sys(t, y, alpha, k0, k1, n, astar):
    a, b = y
    dadt = l(t, alpha, k0, k1, n, astar)*a*(b-1) 
    dbdt = b*(1-k(t, k0, k1, n, astar)*(a+b)-l(t, alpha, k0, k1, n, astar)*a)
    return dadt, dbdt

In [ ]:
# Load and prepare data

filename1 = "6_period.dat"
filename2 = "12_period.dat"
path = "../output/time_series/"

data1 = np.loadtxt(path + filename1)
data2 = np.loadtxt(path + filename2)

start_period = 400

num_of_periods1 = 2
num_of_periods2 = 2

t1 = data1[start_period*100:(start_period+num_of_periods1)*100, 0]
a1 = data1[start_period*100:(start_period+num_of_periods1)*100, 1]
b1 = data1[start_period*100:(start_period+num_of_periods1)*100, 2]
t2 = data2[start_period*100:(start_period+num_of_periods2)*100, 0]
a2 = data2[start_period*100:(start_period+num_of_periods2)*100, 1]
b2 = data2[start_period*100:(start_period+num_of_periods2)*100, 2]

In [ ]:
# Parameters
alpha = 1
k0 = 0.0125
k1_1 = 0.0110
k1_2 = 0.0113
n = 0.5
astar = 40

# Initialize the plot
fig, ax = plt.subplots(figsize=(10, 5))

ax.set_aspect('equal')

# Iterate over the data points
for i in range(0, len(t1), 4):
    # Get the current time and data point
    current_time1 = t1[i]
    current_data_point1 = (a1[i], b1[i])

    # Calculate the vector field at the current data point
    vector_field_1 = sys(current_time1, current_data_point1, alpha, k0, k1_1, n, astar)

    # Plot the vector field at the current data point
    ax.quiver(*current_data_point1, *vector_field_1, color='r', scale = 50, width = 0.003)

for i in range(0, len(t2), 4):
    # Get the current time and data point
    current_time2 = t2[i]
    current_data_point2 = (a2[i], b2[i])

    # Calculate the vector field at the current data point
    vector_field_2 = sys(current_time2, current_data_point2, alpha, k0, k1_2, n, astar)

    # Plot the vector field at the current data point
    ax.quiver(*current_data_point2, *vector_field_2, color='blue', scale = 50, width = 0.003)

# Plot the data points
ax.plot(a1, b1, 'r')
ax.plot(a2, b2, 'b')

plt.title('Vector field of the system of ODEs')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# overlay vector field
k1 = 0.0113

# Create a grid of points
x = np.linspace(30, 50, 50)
y = np.linspace(0, 5, 50)

X, Y = np.meshgrid(x, y)

# Initialize the quiver plot
fig, ax = plt.subplots()
ax.set_aspect('equal')
Q = ax.quiver(X, Y, np.zeros(X.shape), np.zeros(Y.shape), cmap=plt.cm.jet)

# Update function for the animation
def update(num):
    U, V = np.zeros(X.shape), np.zeros(Y.shape)
    NI, NJ = X.shape

    for i in range(NI):
        for j in range(NJ):
            x = X[i, j]
            y = Y[i, j]
            yprime = sys(num, (x, y), alpha, k0, k1, n, astar)
            U[i,j] = yprime[0]
            V[i,j] = yprime[1]

    N = np.sqrt(U**2+V**2)  
    U = U/N
    V = V/N

    Q.set_UVC(U, V)
    return Q,

# Create the animation
ax.plot(a1, b1, 'r')
ax.plot(a2, b2, 'b')
ani = FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi/omega0(k0, astar), 100), blit=True)

plt.title('Vector field of the system of ODEs')
plt.xlabel('x')
plt.ylabel('y')
plt.show()